In [70]:
import pickle
import networkx as nx
import pandas
import pulp
import platform
import numpy as np
import pandas as pd

In [169]:
with open(r"C:\Users\szmat\Documents\GitHub\ExMAS_sideline\Miscellaneous_scripts\data\04-05-23\final_res_04-05-23.obj", 'rb') as file:
    e = pickle.load(file)[0]

In [170]:
requests = e['requests']
rides = e['rides']
len(requests)

198

In [151]:
pairs_graph = nx.Graph()
pairs_graph.add_nodes_from(requests.index)
edges = list()
_rides = rides.copy()
for i, row in _rides.iterrows():
    if len(row.indexes) > 1:
        for j, pax1 in enumerate(row.indexes):
            for k, pax2 in enumerate(row.indexes):
                if pax1 != pax2:
                    edges.append((pax1, pax2, {'u': row.u_pax}))

pairs_graph.add_edges_from(edges)

In [152]:
nx.write_graphml_lxml(pairs_graph, r"C:\Users\szmat\Documents\GitHub\ExMAS_sideline\Miscellaneous_scripts\data\shareability_graph.graphml")

In [171]:
rides.to_excel(r"C:\Users\szmat\Documents\GitHub\ExMAS_sideline\Miscellaneous_scripts\data\rides.xlsx")

In [172]:
requests.to_excel(r"C:\Users\szmat\Documents\GitHub\ExMAS_sideline\Miscellaneous_scripts\data\requests.xlsx")

In [155]:
rides.columns

Index(['indexes', 'u_pax', 'u_veh', 'kind', 'u_paxes', 'times', 'indexes_orig',
       'indexes_dest', 'degree', 'index', 'lambda_r', 'PassHourTrav_ns', 'row',
       'selected'],
      dtype='object')

In [156]:
def match(im, r, share_discount=0.3, matching_obj="u_veh", plot=False, make_assertion=True, logger=None):
    """
    main call of bipartite matching on a graph
    :param im: possible rides
    :param r: requests
    :param params: parameter (including objective function)
    :param plot:
    :param make_assertion: test the results at the end
    :return: rides, secelcted rides, reuests
    """
    request_indexes = dict()
    request_indexes_inv = dict()
    for i, index in enumerate(r.index.values):
        request_indexes[index] = i
        request_indexes_inv[i] = index

    im_indexes = dict()
    im_indexes_inv = dict()
    for i, index in enumerate(im.index.values):
        im_indexes[index] = i
        im_indexes_inv[i] = index

    im['lambda_r'] = im.apply(
        lambda x: share_discount if x.kind == 1 else 1 - x.u_veh / sum([r.loc[_].ttrav for _ in x.indexes]),
        axis=1)

    im['PassHourTrav_ns'] = im.apply(lambda x: sum([r.loc[_].ttrav for _ in x.indexes]), axis=1)

    r = r.reset_index()

    nR = r.shape[0]

    def add_binary_row(r):
        ret = np.zeros(nR)
        for i in r.indexes:
            ret[request_indexes[i]] = 1
        return ret

    logger.info('Matching {} trips to {} rides in order to minimize {}'.format(nR,
                                                                               im.shape[0],
                                                                               matching_obj)) if logger is not None else None
    im['row'] = im.apply(add_binary_row, axis=1)  # row to be used as constrain in optimization
    m = np.vstack(im['row'].values).T  # creates a numpy array for the constrains

    if plot:
        plt.rcParams['figure.figsize'] = [20, 3]
        plt.imshow(m, cmap='Greys', interpolation='nearest')
        # plt.spy(m, c = 'blue')
        plt.show()

    im['index'] = im.index.copy()

    im = im.reset_index(drop=True)

    # optimization
    prob = pulp.LpProblem("Matching problem", pulp.LpMinimize)  # problem

    variables = pulp.LpVariable.dicts("r", (i for i in im.index), cat='Binary')  # decision variables

    cost_col = matching_obj
    if cost_col == 'degree':
        costs = im.indexes.apply(lambda x: -(10 ** len(x)))
    elif cost_col == 'u_pax':
        costs = im[cost_col]  # set the costs
    else:
        costs = im[cost_col]  # set the costs

    prob += pulp.lpSum([variables[i] * costs[i] for i in variables]), 'ObjectiveFun'  # ffef

    j = 0  # adding constrains
    for imr in m:
        j += 1
        prob += pulp.lpSum([imr[i] * variables[i] for i in variables if imr[i] > 0]) == 1, 'c' + str(j)

    solver = pulp.get_solver(solver_for_pulp())
    solver.msg = False
    prob.solve(solver)  # main optimization call
    # prob.solve()  # main optimization call

    logger.info('Problem solution: {}. \n'
                'Total costs for single trips:  {:13,} '
                '\nreduced by matching to: {:20,}'.format(pulp.LpStatus[prob.status], int(sum(costs[:nR])),
                                                          int(pulp.value(
                                                              prob.objective)))) if logger is not None else None

    assert pulp.value(prob.objective) <= sum(costs[:nR]) + 2  # we did not go above original

    locs = dict()
    for variable in prob.variables():
        i = int(variable.name.split("_")[1])

        locs[im_indexes_inv[i]] = (int(variable.varValue))
        # _inData.logger.info("{} = {}".format(int(variable.name.split("_")[1]), int(variable.varValue)))

    return locs

def solver_for_pulp():
    system = platform.system()
    if system == "Windows":
        return "GLPK_CMD"
    else:
        return "PULP_CBC_CMD"

In [163]:
def calculate_results(rides, requests):
    selected = match(rides, requests)
    fin = rides.loc[rides['selected'] == 1]
    return sum(fin["PassHourTrav_ns"]) - sum(fin["u_veh"])

In [164]:
calculate_results(rides, requests)

C:\Users\szmat\.conda\envs\ox\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


38351

In [157]:
selected = match(rides, requests)

C:\Users\szmat\.conda\envs\ox\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [160]:
fin = rides.loc[rides['selected'] == 1]

In [161]:
fin

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,lambda_r,PassHourTrav_ns,row,selected
1,[1],5.984000,440,1,[5.984],"[44, 440]",[1],[1],1.0,1,0.300000,440,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2,[2],2.414700,177,1,[2.4147],"[57, 177]",[2],[2],1.0,2,0.300000,177,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
5,[5],0.748000,55,1,[0.748],"[80, 55]",[5],[5],1.0,5,0.300000,55,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",1
8,[8],2.982900,219,1,[2.9829],"[148, 219]",[8],[8],1.0,8,0.300000,219,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",1
11,[11],3.064500,225,1,[3.0645],"[280, 225]",[11],[11],1.0,11,0.300000,225,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634,"[83, 13, 90, 149]",90.084410,2864,41,"[33.219640752, 24.361776071999998, 20.92171449...","[241.0, 739, 54, 423, 988, 345, 301, 14]","[83, 13, 90, 149]","[149, 90, 13, 83]",NaN,1634,0.590857,7000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
1702,"[73, 78, 89, 125]",34.884841,1490,42,"[12.829838555999999, 5.738204339999999, 12.171...","[795.5, 146, 201, 69, 227, 124, 432, 291]","[73, 78, 89, 125]","[78, 125, 73, 89]",NaN,1702,0.460926,2764,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
1703,"[111, 136, 133, 159]",49.937842,1670,42,"[16.525777536, 9.923199047999999, 14.599155407...","[1212.0, 67, 151, 135, 591, 213, 381, 132]","[111, 136, 133, 159]","[136, 159, 111, 133]",NaN,1703,0.564196,3832,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
1738,"[0, 7, 87, 72, 91]",103.665274,3113,50,"[31.476632747999997, 21.598223675999996, 18.99...","[-155.5, 435, 88, 330, 126, 701, 447, 243, 478...","[0, 7, 87, 72, 91]","[72, 87, 7, 0, 91]",NaN,1738,0.606597,7913,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",1


In [168]:
print(sum(fin["u_veh"]), sum(fin["PassHourTrav_ns"]))

75790 114141


In [167]:
sum(requests['ttrav'])

114141